# Newman 5.12 - The Stefan-Botzmann Constant

If we equate any potential frequency to a spectrum from 0 to infinity (forbidding "negative" frequencies), then each frequency adds to the total amount of energy radiated off the black body per unit area. Additionally, because the denominator contains an exponential with a large number of constant exponents, those portions can be "split" off and moved outside of the frequency dependent portion.

We also convert the limits with a change of variables to allow the computer to do the calculation.

In [1]:
def gaussxw(N,a=None,b=None,verbose=False):
    '''calculate integration points and weights for Gaussian quadrature

    ARGUMENTS:
    N - order of the Gaussian approximation 
    a - lower limit of integral, default a=None (limit = -1)
    b - upper limit of integral, default b=None (limit =  1)
    
    RETURNS
    x,w = integration points x and integration weights w such that 
          sum_i w[i]*f(x[i]) is the Nth-order
          Gaussian approximation to the integral int_{-a}^b f(x) dx
          
    USAGE EXAMPLES:
    def f1(x):
        return x*(x-1)

    N=2
    a=0
    b=1

    #return the correct weights with a single call
    xp,wp = gaussxw(N,a=a,b=b)
    
    #return the weights that may be scaled for different limits (a,b)
    x,w = gaussxw(N)
    xp = 0.5*(b-a)*x + 0.5*(b+a)
    wp = 0.5*(b-a)*w
    
    '''
    
    if verbose==True:
        print("The limits of the function are: ",a,b)
        
    # Initial approximation to roots of the Legendre polynomial
    a1 = np.linspace(3,4*N-1,N)/(4*N+2)
    x = np.cos(np.pi*a1+1/(8*N*N*np.tan(a1)))

    # Find roots using Newton's method
    epsilon = 1e-15
    delta = 1.0
    while delta>epsilon:
        p0 = np.ones(N,float)
        p1 = np.copy(x)
        for k in range(1,N):
            p0,p1 = p1,((2*k+1)*x*p1-k*p0)/(k+1)
        dp = (N+1)*(p0-x*p1)/(1-x*x)
        dx = p1/dp
        x -= dx
        delta = max(abs(dx))

    # Calculate the weights
    w = 2*(N+1)*(N+1)/(N*N*(1-x*x)*dp*dp)
    
    if a==None and b==None:
        #generally if you are recalculating the same integral use this method
        return x,w
    else:
        #if you are only doing a single integral, use this method.
        return 0.5*(b-a)*x + 0.5*(b+a), 0.5*(b-a)*w

In [2]:
import math
import numpy as np

#Function Definitions
def f(x):
    '''The converted integrand for the Stefan-Boltzmann constant calculation
    x: the inputted frequency
    
    thermal: the amount of thermal energy radiated off'''
    
    thermal = ((x**4)*math.exp(x)) / ((math.exp(x)-1)**2)
    return thermal

# Constant and Variable Declaration
k_b = 1.3806*10**(-23) #Botlzmann Constant
h_bar = 1.055*10**(-34) #H-bar in J*s
c = 3*10**8 #Speed of light
T = 273 #Temperature in Kelvin (presumed)
N = 500 #Number of points to be sampled
a = ((5**0.5)-1)/2 #Start point of integral
b = 1 #End point of integral

#Calculations
x,w = gaussxw(N)
xp = 0.5*(b-a)*x + 0.5*(b+a)
wp = 0.5*(b-a)*w

s=0.0
for k in range(N):
    s += wp[k]*f(xp[k])
    
energy = s*(k_b**4)*(T**4)/(4*(math.pi**2)*(c**2)*(h_bar**3))
print(energy)

11.6102388387


I modified the code from Newman 5.9 and adapted changed the internal integrand and outside multiplier. I am unsure of how accurate the answer is, given the semi-random value chosen for T (setting to approximately 0 degrees Celsius).

# Newman 5.19 - Diffraction Gratings

### Part a
If we assume that the function must have zeroes at certain points, then we can also assume that there are discrete u's that result in these zeroes. By solving the equation for u in terms of \alpha, it is revealed that u = \pi / \alpha.

### Part b

In [12]:
import math
import cmath
from scipy import integrate
import numpy as np

#Function definitions
def q(u):
    '''Defines the transmission function
    u: the position along the grating
    '''
    
    return math.sin((math.pi/(20*10**-6))*u)**2

### Part c

In [28]:
import matplotlib.pylab as plt
%matplotlib inline

# Function Definition
def angle_coefficient (h, x, wavelength, focal):
    return 2*math.pi*h*x/(wavelength*focal)

def Eueler_Conversion (coefficient):
    return float(math.cos(coefficient)+float(math.sin(coefficient)))

def coefficientDeterminatorV2(step,max,order):
    """Determines the coefficient for up to quartic integrations"""
    """Coefficient Determinator 2: Electric Boogaloo"""
    if (order==1):
        denom=0.5
        if (step==0 or step==max-1):
            num=1
        else:
            num=2
    elif(order==2):
        denom=3
        if(step==0 or step==max-1):
            num=1
        elif (step%2==0):
            num=2
        else:
            num=4
    elif(order==3):
        denom=8
        if(step==0 or step==max-1):
            num=3
        elif(step%2==0):
            num=9
        else:
            num=6
    elif(order==4):
        denom=15
        if(step==0 or step==max-1):
            num=14/3
        elif(step%2==0):
            num=64/3
        elif(step%4==1):
            num=28/3
        else:
            num=8
    else:
        print("Please enter an order between 0 and 4")
    return num/denom

def integrate (f,a,b,N,order):
    """Integrates a function given the function, its limits, the number of steps desired, and the order of fit desired"""
    h=(b-a)/N
    i=s=0
    for i in range(N):
        s+= coefficientDeterminatorV2(i,N,order) * f(a+i*h) * h
    return s


# Variable and Constant Declaration
wavelength = 500*10**-9 # Wavelength in m
focal = 1 # Focal length in m
screen = 0.1 # Screen length in m
grating = 200*10**-6 # Grating width in m
N = 50 #Number of runs
h = np.arange (-screen, screen, N)
order =2
Pattern = []

# Function runs
for i in range (len(h)):
    Pattern.append(math.fabs(integrate(q(x) * Eueler_Conversion(angle_coefficient(h[i],x,wavelength,focal)), -screen/2, screen/2, N, order))**2)


plt.plot(h,Pattern, label = "Intensity Pattern")
plt.show()

TypeError: 'float' object is not callable